# Lab 2b

This lab is for practicing Python’s <i>generators</i> and <i>streaming</i>. We’re going to use the Citibike dataset posted on NYU Classes.



## Task 1

Your task is to <b>compute the median age</b> of the Citibike’s <b>subscribed</b> customers, aka. usertype is "Subscriber". You are required to read data line by line and are not allowed to store the entire data set in memory. Indeed, you should not have any containers (e.g. list, dictionary, DataFrame, etc.) with more than a handful, e.g. strictly < 100, of elements in memory either as a local or a global variable. You can use the citibike.csv data file that we have on NYU Classes for testing, but we will evaluate your code on a much larger input to ensure it’s streaming capability.

The code block below is taken mostly from our lab and would stream data from the citibike.csv for on-demand processing. The data file should be stored on the same folder with this notebook. You will need to replace the portion inside the for loop with your own code.

In [1]:
import pandas as pd
df = pd.read_csv('citibike.csv')
df.head()

,cartodb_id,the_geom,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,1,NaN,801,2015-02-01 00:00:00+00,2015-02-01 00:14:00+00,521,8 Ave & W 31 St,40.750450,-73.994811,423,W 54 St & 9 Ave,40.765849,-73.986905,17131,Subscriber,1978.0,2
1,2,NaN,379,2015-02-01 00:00:00+00,2015-02-01 00:07:00+00,497,E 17 St & Broadway,40.737050,-73.990093,504,1 Ave & E 15 St,40.732219,-73.981656,21289,Subscriber,1993.0,1
2,3,NaN,2474,2015-02-01 00:01:00+00,2015-02-01 00:42:00+00,281,Grand Army Plaza & Central Park S,40.764397,-73.973715,127,Barrow St & Hudson St,40.731724,-74.006744,18903,Subscriber,1969.0,2
3,4,NaN,818,2015-02-01 00:01:00+00,2015-02-01 00:15:00+00,2004,6 Ave & Broome St,40.724399,-74.004704,505,6 Ave & W 33 St,40.749013,-73.988484,21044,Subscriber,1985.0,2
4,5,NaN,544,2015-02-01 00:01:00+00,2015-02-01 00:10:00+00,323,Lawrence St & Willoughby St,40.692362,-73.986317,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,19868,Subscriber,1957.0,1


In [2]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row
            
### NOTE: You can initialize any global variables below,
### but they should hold less than 100 elements.

count = {}

for row in csvRows('citibike.csv'):
    ### NOTE: perform your streaming computation here. 'row' is a
    ### tuple of values for the current record of the input file.
    ### You should replace 'pass' below with your code.
    if row['usertype'] == 'Subscriber':
        age = 2018-int(row['birth_year'])
        count[age] = count.get(age, 0)+1

### NOTE: You can do further processing to get the median age value
###
medianAge = 0
medianCount = 0
totalCount = sum(count.values())
for age, c in sorted(count.items()):
    medianCount += c
    if medianCount >= totalCount/2:
        medianAge = age
        break

###
print medianAge

41


## Task 2

Your task is to write a generator to extract the first ride of the day from a Citibike data stream. The data stream is sorted based on starting times (similar to the <b>citibike.csv</b> file uploaded on NYU Classes). The first ride of the day is interpreted as the ride with the earliest starting time of a day. For the sample data, which is a week worth of citibike records, your generator should only generate 7 items (one for each day).

You are given a template with the sample generator <b>firstRide</b>. The generator currently takes in <b>csv.DictReader</b> generator and output its first element. Please adjust this generator to output the first ride of the day for the entire stream as specified above. The output of the generator must be in the same format as csv.DictReader. You can think of this generator as a filter only passing certain records out. \

In [3]:
import csv
import datetime

### NOTE: You need to change the body of the generator firstRide
### in order to output trip record that appeared first in each day
### using the same dict format as csv.DictReader.

def firstRide(reader):
    lastDay = None
    for row in reader:
        current = row['starttime'].split()[0]
        if current != lastDay: #this is first of day:
            yield row
            lastDay = current

### NOTE: You SHOULD NOT modify the code below. If you
### write your firstRide generator above correctly, the
### code below will output the correct information

with open('citibike.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    for row in firstRide(reader):
        print ','.join(map(row.get, reader.fieldnames))


1,,801,2015-02-01 00:00:00+00,2015-02-01 00:14:00+00,521,8 Ave & W 31 St,40.75044999,-73.99481051,423,W 54 St & 9 Ave,40.76584941,-73.98690506,17131,Subscriber,1978,2
6442,,199,2015-02-02 00:02:00+00,2015-02-02 00:05:00+00,442,W 27 St & 7 Ave,40.746647,-73.993915,489,10 Ave & W 28 St,40.75066386,-74.00176802,20684,Subscriber,1992,1
7901,,704,2015-02-03 00:00:00+00,2015-02-03 00:12:00+00,387,Centre St & Chambers St,40.71273266,-74.0046073,2008,Little West St & 1 Pl,40.70569254,-74.01677685,20328,Subscriber,1982,1
12655,,146,2015-02-04 00:00:00+00,2015-02-04 00:02:00+00,237,E 11 St & 2 Ave,40.73047309,-73.98672378,438,St Marks Pl & 1 Ave,40.72779126,-73.98564945,15253,Subscriber,1969,1
21628,,1034,2015-02-05 00:00:00+00,2015-02-05 00:17:00+00,497,E 17 St & Broadway,40.73704984,-73.99009296,461,E 20 St & 2 Ave,40.73587678,-73.98205027,20290,Subscriber,1971,1
30836,,212,2015-02-06 00:01:00+00,2015-02-06 00:05:00+00,491,E 24 St & Park Ave S,40.74096374,-73.98602213,472,E 32 St & Park Ave,40